请放到markdown文件夹下面使用~

In [10]:
import os
import re
import requests

def download_image(url, folder, counter):
    # 确保文件夹存在
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    # 从URL获取图片内容
    response = requests.get(url)
    if response.status_code == 200:
        # 构造文件名，添加序号以避免覆盖
        filename = os.path.join(folder, f'image{counter:03d}.jpeg')
        
        # 保存图片
        with open(filename, 'wb') as f:
            f.write(response.content)
        return filename
    else:
        print("Failed to download image: Status code", response.status_code)
        return None

def extract_title_from_md_path(md_file_path):
    # 从Markdown文件路径中提取标题
    base_name = os.path.basename(md_file_path)
    return os.path.splitext(base_name)[0]  # 移除扩展名

def replace_image_links_in_md(md_file, img_folder):
    # 提取文章标题
    title = extract_title_from_md_path(md_file)
    # 创建文章的独立图片文件夹
    article_img_folder = os.path.join(img_folder, title)
    
    # 读取Markdown文件
    with open(md_file, 'r', encoding='utf-8') as f:
        md_content = f.read()
    
    # 找到所有图片链接，包括带或不带替代文本的情况
    img_links = re.findall(r'!\[(.*?)\]\((https?://[^\)]+)\)', md_content)
    
    # 初始化计数器
    image_counter = 0

    # 替换链接
    for alt_text, link in img_links:
        # 下载图片并获取本地路径
        local_path = download_image(link, article_img_folder, image_counter)
        if local_path:
            # 构造新的Markdown链接，如果alt_text为空则使用文章标题
            new_alt_text = alt_text if alt_text else title
            # 构造相对路径
            relative_path = os.path.relpath(local_path, img_folder)
            new_link = f"![{new_alt_text}](./img/{relative_path})"
            # 替换链接
            md_content = md_content.replace(f'![{alt_text}]({link})', new_link)
        image_counter += 1  # 确保每次迭代计数器都递增
    
    # 写回Markdown文件
    with open(md_file, 'w', encoding='utf-8') as f:
        f.write(md_content)

# 使用示例
md_file_path = './20240120哪个城市是中央眼中的心头爱基于新闻联播文本的大数据分析.md'  # 你的Markdown文件路径
img_folder_path = './img'  # 图片保存的文件夹路径

replace_image_links_in_md(md_file_path, img_folder_path)